In [139]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import train_test_split

In [172]:
df = pd.read_csv('misconduct.csv', header=False)
df = df.dropna(subset=['Misconduct Description'])
df = df.dropna(subset=['Action'])

In [173]:
# df[df.Address.str.contains('NY')]
df = df[pd.isnull(df['Misconduct Description']) == False]

In [174]:
df

,Name,Address,License Number,License Type,Year of Birth,Effective Date,Action,Misconduct Description,Date Updated
0,"563 Grand Medical, P.C.","563 Grand Street\r\nBrooklyn, NY 11211",196275,NaN,NaN,09/29/2010,Revocation of certificate of incorporation.,The corporation admitted guilt to the charge o...,09/29/2010
1,"Aaron, Joseph","2803 North 700 East \r\n Provo, Utah 84604",072800,MD,1927,01/13/1999,License Surrender,This action modifies the penalty previously im...,NaN
2,"Aarons, Mark Gold","P.O.Box 845\r\nSouthern Pines, North Carolina ...",161530,MD,1958,12/13/2005,License limited until the physician's North Ca...,The physician did not contest the charge of ha...,12/06/2005
3,"Abadi, Jamsheed S",Address redacted,136045,MD,1939,08/14/2013,License surrender.,The physician did not contest the charge of fa...,08/07/2013
4,"Abassi, Lila",Address redacted,None,MD,1978,08/12/2014,Interim non-disciplinary order of conditions p...,This action is not disciplinary in nature.,08/06/2014
5,"Abbasi, Abdul Hafeez","237 Travis Avenue\r\nElmont, New York 11003",183025,MD,1955,04/13/2004,The New York State Board of Regents restored t...,"Previously the Review Board on November 14,199...",04/15/2004
6,"Abbassi, Samih R","11 Gorga Place \r\n Washington Township, New J...",171180,MD,NaN,04/29/1996,"License surrender, $5,000 fine",The physician did not contest the charges of n...,06/21/2001
7,"Abboud, Aiman Michael",1009 Brighton Beach Avenue\r\nSuite 3\r\nBrook...,232699,DO,1962,12/15/2014,The physician is subject to a license limitati...,The physician asserted he could not successful...,12/08/2014
8,"Abdel-Al, Naglaa Zidan Elsayed",Address redacted,227440,MD,1969,10/03/2011,License surrender,The physician did not contest the charge of ha...,09/26/2011
9,"Abdel-Hameed, Mohammad Fathi Ahmad","10150 Brandon Circle \r\n Orlando, Florida 32836",173309,MD,1938,04/11/2001,No action taken against the physician's licens...,The Hearing Committee sustained the charge fin...,06/21/2001


In [175]:
for x in ['surrender', 'revoked','revoke', 'surrendered',
          'Surrender', 'Revoked', 'Revoke', 'Revocation', 'revocation',
          'Surrendered']:
    df.loc[df['Action'].str.contains(x), 'Action'] = 'lost license'


In [176]:
for x in ['dissmissed', 'dropped', 'Dismissed', 'Dropped', 'No penalty']:
    df.loc[df.Action.str.contains(x), 'Action'] = 'dropped'

In [177]:
for x in ['fine', 'Fined', 'Fine', 'fined']:
    df.loc[df.Action.str.contains(x), 'Action'] = 'fine'

In [178]:
for x in ['Suspension', 'suspension']:
    df.loc[df.Action.str.contains(x), 'Action'] = 'suspension'

In [179]:
df.loc[df.Action == 'fine']

,Name,Address,License Number,License Type,Year of Birth,Effective Date,Action,Misconduct Description,Date Updated
11,"Abdeljaber, Mutee Husein",Address Redacted,189612,MD,1957,06/19/2008,fine,The physician did not contest the charge of ha...,06/12/2008
19,"Abir, Fereydoon","675 Water Street \r\n New York, New York 10002",121310,MD,1941,12/11/1998,fine,The Hearing Committee sustained the charge fin...,06/21/2001
29,"Abramson, David","4200 42nd Street, NW \r\n Washington, D.C. ...",172405,MD,NaN,01/20/1997,fine,The Hearing Committee sustained the charge fin...,NaN
34,"Acenas-Recientes, Elizabeth Sison","150 Robby Lane\r\nManhasset Hills, New York 11040",163510,MD,1950,11/17/2004,fine,The physician did not contest the charge of ha...,11/10/2004
37,"Acker, Gerald Alan","1 Expressway Plaza \r\n Roslyn Heights, New Yo...",094882,MD,NaN,07/22/1992,fine,The physician admitted to having practiced the...,NaN
44,"Acuna, Jose Gonzalez","116 Weldon Way \r\n Pennington, New Jersey 08...",190863,MD,NaN,11/11/1997,fine,The physician did not contest the charges of m...,NaN
51,"Adams, Peter","265 Mountain Road\r\nEngle Wood, New Jersey 07631",108609,MD,1944,04/24/2000,fine,The Hearing Committee sustained the charges fi...,NaN
55,"Adegbite, Samson","28 Stoneledge Court \r\n Williamsville, New Yo...",175350,MD,1952,07/08/1998,fine,The physician did not contest the charges of c...,NaN
56,"Adegbite, Samson","28 Stoneledge Court \r\n Williamsville, New Yo...",175350,MD,1952,09/09/1999,fine,The physician admitted to the charge of conduc...,NaN
74,"Africano, Enrique A","240 N. Wickham Road, Suite 104\r\nMelbourne, F...",192924,MD,1951,11/01/2010,fine,The physician did not contest the charge of ha...,10/25/2010


In [180]:
df.loc[df.Action.str.len() > 12, 'Action'] = 'reprimand'
df.loc[df.Action == 'Reprimand', 'Action'] = 'reprimand'

In [181]:
df.iloc[15]['Misconduct Description']

'The Hearing Committee sustained the charges finding the physician guilty of having been convicted of Obstruction of Criminal Investigations of Health Care Offenses in United States District Court, Massachusetts and having been disciplined by the Massachusetts Board of Registration in Medicine.'

In [182]:
pd.unique(df.Action)

array(['lost license', 'reprimand', 'fine', 'suspension', 'dropped'], dtype=object)

In [183]:
total = 0
for x in pd.unique(df.Action):
    print '{}: {}'.format(x, len(df[df.Action == x]))
    total += len(df[df.Action == x])
print total

lost license: 3432
reprimand: 1722
fine: 885
suspension: 1020
dropped: 41
7100


In [184]:
X_full = list(df['Misconduct Description'])
y_full = list((df['Action'])

SyntaxError: invalid syntax (<ipython-input-184-ca2f7c93af18>, line 2)

In [192]:
#train test splitting

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, train_size=.8, random_state=19)

In [193]:
full_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier())])

In [195]:
full_clf.fit(X_train, y_train)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [191]:
(df['Misconduct Description'])

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''